# Setup

### Import libraries

In [1]:
import os
import math
from collections import defaultdict
import gc

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from tqdm import tqdm

### Set seed

In [2]:
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Data

### Load file

In [7]:
path = '/data/ephemeral/home/data/cold/'
mode = 5

full_df = pd.read_csv(path + 'full.csv')
train_df = pd.read_csv(path + f'{mode}shot/train_{mode}.csv')

val_k = pd.read_csv(path + f'{mode}shot/val_{mode}_k.csv')
test_k = pd.read_csv(path + f'{mode}shot/test_{mode}_k.csv')

val_n = pd.read_csv(path + f'{mode}shot/val_{mode}_n.csv')
test_n = pd.read_csv(path + f'{mode}shot/test_{mode}_n.csv')

### Reindex dataframe

In [8]:
users = full_df['user_id'].unique()
items = full_df['item_id'].unique()

user2idx = {user: i for i, user in enumerate(users)}
item2idx = {item: i for i, item in enumerate(items)}

In [9]:
# 리인덱싱 수행
train_df['user_id'] = train_df['user_id'].map(user2idx)
train_df['item_id'] = train_df['item_id'].map(item2idx)

val_k['user_id'] = val_k['user_id'].map(user2idx)
val_k['item_id'] = val_k['item_id'].map(item2idx)

test_k['user_id'] = test_k['user_id'].map(user2idx)
test_k['item_id'] = test_k['item_id'].map(item2idx)

val_n['user_id'] = val_n['user_id'].map(user2idx)
val_n['item_id'] = val_n['item_id'].map(item2idx)

test_n['user_id'] = test_n['user_id'].map(user2idx)
test_n['item_id'] = test_n['item_id'].map(item2idx)

In [10]:
n_users = len(users)
n_items = len(items)

del full_df

In [11]:
val_users = val_k['user_id'].unique()
test_users = test_k['user_id'].unique()

In [12]:
train_user_pos_items = {k: list(v['item_id'].values) for k, v in train_df.groupby('user_id')}
val_user_pos_items = {k: list(v['item_id'].values) for k, v in val_k.groupby('user_id')}
test_user_pos_items = {k: list(v['item_id'].values) for k, v in test_k.groupby('user_id')}

val_actual = {k: list(v['item_id'].values) for k, v in val_n.groupby('user_id')}
test_actual = {k: list(v['item_id'].values) for k, v in test_n.groupby('user_id')}

In [16]:
del val_k, test_k, val_n, test_n

### Create COO

In [17]:
def create_coo_matrix(df, n_users, n_items):
    # torch 텐서로 변환
    user_id_tensor = torch.tensor(df['user_id'].values, dtype=torch.long)
    item_id_tensor = torch.tensor(df['item_id'].values, dtype=torch.long)
    label_tensor = torch.ones(len(df), dtype=torch.float32)

    # COO 희소 텐서 생성
    indices = torch.stack([user_id_tensor, item_id_tensor])
    values = label_tensor
    size = (n_users, n_items)  # 전체 유저 x 전체 아이템 크기로 지정

    return torch.sparse_coo_tensor(indices, values, size)

In [18]:
train_coo = create_coo_matrix(train_df, n_users, n_items)

### Dataset Class

In [19]:
class BPRDataset(Dataset):
    '''
    args:
        coo: torch.sparse.coo_tensor, user-item interaction matrix
        train: bool, train or test
        n_negs: int, number of negative samples
    '''
    def __init__(self, coo, user_pos_items, n_negs=1):
        self.users, self.pos_items = coo._indices()
        self.user_pos_items = user_pos_items
        self.n_negs = n_negs
        self.n_items = coo.shape[1]
        self.n_inter = coo._values().shape[0]

    def __len__(self):
        return self.n_inter

    def __getitem__(self, idx):
        user = self.users[idx]
        pos_item = self.pos_items[idx]
        neg_item = self._get_neg_item(user)
        return user, pos_item, neg_item
    
    # def _get_neg_items(self, user):
    #     neg_items = set()
    #     while len(neg_items) < self.n_negs:
    #         neg_item = torch.randint(0, self.n_items, (1,)).item()
    #         if neg_item not in self.user_pos_items[user]:
    #             neg_items.add(neg_item)
    #     return torch.LongTensor(list(neg_items))
    
    def _get_neg_item(self, user):
        user_pos_items = set(self.user_pos_items[user])
        neg_item = torch.randint(0, self.n_items)
        while neg_item in user_pos_items:
            neg_item = torch.randint(0, self.n_items)
        return neg_item


### DataLoader

In [20]:
train_dataset = BPRDataset(train_coo, train_user_pos_items)

In [21]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)

In [22]:
del train_df

In [23]:
# for user, pos_item, neg_items in train_loader:
#     print(user.shape)
#     print(pos_item.shape)
#     print(neg_items.shape)
#     print(neg_items[:, 0].shape)
#     break

## Model

In [24]:
class NCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim=64, dropout=0.2):
        super(NCF, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.dropout = dropout

        self.user_emb = nn.Embedding(n_users, emb_dim)
        self.item_emb = nn.Embedding(n_items, emb_dim)
        self.mlp = nn.Sequential(  # [batch_size, emb_dim * 2]
            nn.Dropout(dropout),
            nn.Linear(emb_dim * 2, emb_dim),  # [batch_size, emb_dim]
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(emb_dim, emb_dim // 2),  # [batch_size, emb_dim // 2]
            nn.ReLU(),
            nn.Linear(emb_dim // 2, 1)  # [batch_size, 1]
        )

        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_normal_(self.user_emb.weight)
        nn.init.xavier_normal_(self.item_emb.weight)
        for layer in self.mlp:
            if isinstance(layer, nn.Linear):
                nn.init.kaiming_normal_(layer.weight)
                nn.init.constant_(layer.bias, 0.1)

    def forward(self, user, item):
        user_emb = self.user_emb(user)
        item_emb = self.item_emb(item)
        concat = torch.cat([user_emb, item_emb], dim=-1)  # [batch_size, emb_dim * 2]
        return self.mlp(concat).squeeze()  # [batch_size]

### BPR loss

In [25]:
# BPR 손실 함수
class BPRLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pos_scores, neg_scores):
        return -torch.log(torch.sigmoid(pos_scores - neg_scores)).mean()

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
emb_dim = 64
dropout = 0.2
model = NCF(n_users, n_items, emb_dim, dropout)
model.to(device)
criterion = BPRLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [38]:
epochs = 10
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for user, pos_item, neg_item in tqdm(train_loader):
        user = user.to(device)
        pos_item = pos_item.to(device)
        neg_item = neg_item.to(device)

        pos_scores = model(user, pos_item)
        neg_scores = model(user, neg_item)

        loss = criterion(pos_scores, neg_scores)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    print(f'Epoch {epoch + 1}, Loss: {train_loss / len(train_loader):.4f}')

Epoch 1, Loss: 0.1224
Epoch 2, Loss: 0.1054
Epoch 3, Loss: 0.0966
Epoch 4, Loss: 0.0932
Epoch 5, Loss: 0.0904
Epoch 6, Loss: 0.0873
Epoch 7, Loss: 0.0848
Epoch 8, Loss: 0.0830
Epoch 9, Loss: 0.0817
Epoch 10, Loss: 0.0810


In [39]:
save_dir = 'saved/'
os.makedirs(save_dir, exist_ok=True)

model_path = save_dir + 'ncf.pth'
torch.save(model.state_dict(), model_path)

print(f'Model saved to {model_path}')

In [27]:
model.load_state_dict(torch.load('../saved/ncf.pth', weights_only=True))
model.eval()

NCF(
  (user_emb): Embedding(137534, 64)
  (item_emb): Embedding(24799, 64)
  (mlp): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=128, out_features=64, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [33]:
import torch
import numpy as np
import bottleneck as bn
from tqdm import tqdm

def recommend_items(model, user_id, num_items, top_k=10, train_items=None):
    """
    단일 유저에 대해 전체 아이템에 대한 스코어를 계산한 후,
    이미 학습에 활용된 아이템(train_items)이 있을 경우 이를 마스킹(-무한대로 대체)하고,
    bn.argpartition을 활용해 상위 top_k 아이템을 효율적으로 추출하는 함수.
    
    인자:
        model: user_id와 item_id의 텐서를 입력받아 스코어를 반환하는 추천 모델.
        user_id (int): 추천을 위한 대상 유저 ID.
        num_items (int): 전체 아이템의 개수.
        top_k (int): 추천할 아이템 수.
        train_items (list 또는 np.array, optional): 학습 시 활용된 해당 유저의 아이템 인덱스 리스트.
    
    반환:
        추천 아이템 인덱스 리스트 (정렬되어 있음)
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # user_id를 전체 아이템 개수만큼 반복하여 텐서 생성
    user_ids = torch.full((num_items,), user_id, dtype=torch.long).to(device)
    # 모든 아이템의 인덱스 생성
    item_ids = torch.arange(num_items, dtype=torch.long).to(device)
    
    with torch.no_grad():
        scores = model(user_ids, item_ids)
    
    # train_items가 제공되면 해당 아이템의 스코어를 마스킹 처리 (-무한대값으로 대체)
    if train_items is not None:
        # torch indexing은 list나 array로도 발동됨
        scores[train_items] = -float('inf')
    
    # GPU에 있을 경우 CPU로 옮긴 후 numpy 배열로 변환
    scores_np = scores.cpu().numpy()
    
    # bottleneck의 argpartition을 사용하여 상위 top_k의 후보 인덱스를 추출
    # 음수 부호를 취해 내림차순 정렬 효과를 냄.
    candidate_indices = bn.argpartition(-scores_np, top_k-1)[:top_k]
    
    # argpartition은 정렬되어 있지 않으므로, 위 후보들에 대해 추가 정렬(내림차순) 수행
    sorted_top_indices = candidate_indices[np.argsort(-scores_np[candidate_indices])]
    
    return sorted_top_indices.tolist()

In [39]:
val_recommendations = {user_id: recommend_items(model, user_id, n_items, 10, val_user_pos_items[user_id]) for user_id in tqdm(val_users)}

100%|██████████| 13753/13753 [00:22<00:00, 624.03it/s]


In [68]:
# class AutoRec(nn.Module):
#     def __init__(self, num_users, num_items, hidden_units=256):
#         super(AutoRec, self).__init__()
#         self.user_embedding = nn.Embedding(num_users, hidden_units)
#         self.item_embedding = nn.Embedding(num_items, hidden_units)
#         self.fc1 = nn.Linear(hidden_units, hidden_units)
#         self.fc2 = nn.Linear(hidden_units, 1)
#         self.relu = nn.ReLU()

#     def forward(self, user_ids, item_ids):
#         user_embeds = self.user_embedding(user_ids)
#         item_embeds = self.item_embedding(item_ids)
#         x = user_embeds * item_embeds
#         x = self.relu(self.fc1(x))
#         return self.fc2(x).squeeze()

In [45]:
def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = set(predicted[:k])
    return len(actual_set & predicted_at_k) / k

def precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = set(predicted[:k])
    return len(actual_set & predicted_at_k) / k

def ndcg_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    dcg = sum([1 / np.log2(i + 2) for i, p in enumerate(predicted_at_k) if p in actual_set])
    idcg = sum([1 / np.log2(i + 2) for i in range(min(len(actual_set), k))])
    return dcg / idcg if idcg > 0 else 0.0

def mean_average_precision_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_at_k = predicted[:k]
    ap_sum = sum([(i + 1) / (idx + 1) for idx, i in enumerate(predicted_at_k) if i in actual_set])
    return ap_sum / k


In [42]:
results = list()
for user in val_users:
    results.append(recall_at_k(val_actual[user], val_recommendations[user], 10))
sum(results) / len(val_users)

0.07550352650331352

In [43]:
results = list()
for user in val_users:
    results.append(ndcg_at_k(val_actual[user], val_recommendations[user], 10))
sum(results) / len(val_users)

0.1277575648681164

In [44]:
results = list()
for user in val_users:
    results.append(mean_average_precision_at_k(val_actual[user], val_recommendations[user], 10))
sum(results) / len(val_users)

2.534017804484793

In [ ]:
# metrics = {}
# for user in val_users:
#     metrics['Recall@10'].append(recall_at_k(val_actual[user], val_recommendations[user], 10))
# #     metrics['Precision@10'] = precision_at_k(val_actual[user], val_recommendations[user], 10)
# #     metrics['NDCG@10'] = ndcg_at_k(val_actual[user], val_recommendations[user], 10)
# #     metrics['MAP@10'] = mean_average_precision_at_k(val_actual[user], val_recommendations[user], 10)
# metrics

In [39]:

def evaluate_model(model, test_loader, k_values):
    model.eval()
    results = {k: {"recall": [], "precision": [], "ndcg": [], "map": []} for k in k_values}
    
    with torch.no_grad():
        for user_ids, item_ids, ratings in tqdm(test_loader, desc="Evaluating"):
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            predictions = model(user_ids, item_ids)
            
            # 각 사용자에 대해 평가
            for user, items, preds in zip(user_ids, item_ids, predictions):
                actual = items[ratings == 1].tolist()
                predicted = items[torch.argsort(preds, descending=True)].tolist()
                
                for k in k_values:
                    results[k]["recall"].append(normalized_recall_at_k(actual, predicted, k))
                    results[k]["precision"].append(normalized_precision_at_k(actual, predicted, k))
                    results[k]["ndcg"].append(ndcg_at_k(actual, predicted, k))
                    results[k]["map"].append(mean_average_precision_at_k(actual, predicted, k))

    
    # 결과 집계
    aggregated_results = {
        k: {metric: np.mean(values) for metric, values in results[k].items()}
        for k in k_values
    }
    
    return aggregated_results
